In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import tarfile

data_dir = '/content/drive/MyDrive/심층신경망개론/'
images_tar_path = os.path.join(data_dir, '102flowers.tgz')
extracted_dir = os.path.join(data_dir, 'jpg')


if not os.path.exists(extracted_dir):
    print("이미지 데이터 추출 중...")
    with tarfile.open(images_tar_path, 'r:gz') as tar:
        tar.extractall(path=data_dir)
    print("이미지 데이터 추출 완료.")
else:
    print("이미지 데이터가 이미 추출되어 있습니다.")


import scipy.io
import pandas as pd
from sklearn.model_selection import train_test_split
from shutil import copyfile

labels_mat_path = os.path.join(data_dir, 'imagelabels.mat')
labels_mat = scipy.io.loadmat(labels_mat_path)
labels = labels_mat['labels'][0]


setid_mat_path = os.path.join(data_dir, 'setid.mat')
setid_mat = scipy.io.loadmat(setid_mat_path)
train_ids = setid_mat['trnid'][0] - 1
val_ids = setid_mat['valid'][0] - 1
test_ids = setid_mat['tstid'][0] - 1


image_dir = extracted_dir
image_files = sorted([f for f in os.listdir(image_dir) if f.endswith('.jpg')])


df = pd.DataFrame({
    'filename': image_files,
    'label': labels
})

# 학습, 검증, 테스트 세트 분할
train_df = df.iloc[train_ids].reset_index(drop=True)
val_df = df.iloc[val_ids].reset_index(drop=True)
test_df = df.iloc[test_ids].reset_index(drop=True)

print(f"학습 세트 크기: {len(train_df)}")
print(f"검증 세트 크기: {len(val_df)}")
print(f"테스트 세트 크기: {len(test_df)}")




# 폴더
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')
test_dir = os.path.join(data_dir, 'test')

for directory in [train_dir, val_dir, test_dir]:
    os.makedirs(directory, exist_ok=True)

    for label in df['label'].unique():
        os.makedirs(os.path.join(directory, str(label)), exist_ok=True)


def copy_images(df, target_dir):
    for _, row in df.iterrows():
        src = os.path.join(image_dir, row['filename'])
        dst = os.path.join(target_dir, str(row['label']), row['filename'])
        copyfile(src, dst)

# 이미지 복사
print("학습 이미지 복사 중...")
copy_images(train_df, train_dir)
print("검증 이미지 복사 중...")
copy_images(val_df, val_dir)
print("테스트 이미지 복사 중...")
copy_images(test_df, test_dir)
print("모든 이미지 복사 완료.")



!pip install timm
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from sklearn.metrics import accuracy_score, precision_score, recall_score

# 하이퍼파라미터
learning_rate = 1e-4
num_epochs = 10
num_classes = 102
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from timm import create_model

# 모델 불러오기
model = create_model('swin_tiny_patch4_window7_224', pretrained=True, num_classes=num_classes)
model = model.to(device)

# 손실함수, 옵티마이져 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')
test_dir = os.path.join(data_dir, 'test')

# 데이터 변환
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
}

# 데이터셋 정의
datasets = {
    'train': datasets.ImageFolder(train_dir, transform=data_transforms['train']),
    'val': datasets.ImageFolder(val_dir, transform=data_transforms['val']),
    'test': datasets.ImageFolder(test_dir, transform=data_transforms['test']),
}

# DataLoader 생성
dataloaders = {
    x: DataLoader(datasets[x], batch_size=32, shuffle=(x == 'train'), num_workers=2)
    for x in ['train', 'val', 'test']
}

# train
def train_model(model, dataloaders, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        print("-" * 20)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()

                #순전파
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    #역전파
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

    print("Training complete.")
    return model

model = train_model(model, dataloaders, criterion, optimizer, num_epochs)

# 모델 평가
def evaluate_model(model, dataloader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')

    return accuracy, precision, recall


accuracy, precision, recall = evaluate_model(model, dataloaders['test'])
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")


pip install torchprofile
from torchprofile import profile_macs
dummy_input = torch.randn(1, 3, 224, 224).to(device)
flops = profile_macs(model, dummy_input)
print(f"FLOPs: {flops / 1e9:.4f} GFLOPs")